# W01 · Deep Learning Foundations

This week introduces the basic building blocks of deep learning with tensors, gradients, and a tiny neural network.

**Ahmed Métwalli**

## Objectives

- Understand tensors and automatic differentiation at a high level
- Learn mean squared error (MSE)
- Train a tiny dense model on Fashion-MNIST
- Log training to TensorBoard

In [ ]:
# Basic setup and reproducibility
import tensorflow as tf, keras, random, numpy as np
print("TF:", tf.__version__, "| Keras:", keras.__version__)
print("Devices:", tf.config.list_physical_devices())

SHOW_SOLUTIONS = False

SEED = 2025
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

## Very Short Concepts

A tensor is a generalization of scalars, vectors, and matrices.

Automatic differentiation computes gradients of a loss with respect to model weights.

## Math: MSE and gradients

The mean squared error compares predictions with targets.

$$\text{MSE}(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

For $\hat{y}_i = w x_i + b$ the gradients become

$$\frac{\partial L}{\partial w} = -\frac{2}{N} \sum_{i} x_i\,(y_i - (w x_i + b))$$

$$\frac{\partial L}{\partial b} = -\frac{2}{N} \sum_{i} (y_i - (w x_i + b))$$

In [ ]:
# Worked Example A: Closed-form line fit with NumPy
import numpy as np
import matplotlib.pyplot as plt

# Generate simple data with noise around a line
x = np.linspace(0.0, 5.0, 50)
noise = 0.5 * np.random.randn(50)
y = 2.5 * x + 1.0 + noise

# Estimate slope and intercept using covariance/variance intuition
x_mean = x.mean()
y_mean = y.mean()
cov_xy = np.mean((x - x_mean) * (y - y_mean))
var_x = np.mean((x - x_mean) ** 2)
slope = cov_xy / var_x
intercept = y_mean - slope * x_mean
print(f"Estimated slope: {slope:.3f}, intercept: {intercept:.3f}")

# Predictions from the fitted line
y_hat = slope * x + intercept

plt.figure(figsize=(6, 4))
plt.scatter(x, y, label="data")
plt.plot(x, y_hat, color="crimson", label="fitted line")
plt.title("Closed-form line fit")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Worked Example B: GradientTape mini-demo
import matplotlib.pyplot as plt
import tensorflow as tf, numpy as np

n = 256
x = tf.linspace(-2.0, 2.0, n)[:, None]
true_w, true_b = 3.0, -0.7
y = true_w * x + true_b + 0.3 * tf.random.normal(shape=x.shape, seed=SEED)

w = tf.Variable(tf.random.normal([1,1], stddev=0.1, seed=SEED))
b = tf.Variable(tf.zeros([1]))
opt = tf.keras.optimizers.SGD(learning_rate=0.1)

losses = []
for step in range(200):
    with tf.GradientTape() as tape:
        y_hat = x @ w + b
        loss = tf.reduce_mean((y - y_hat) ** 2)
    grads = tape.gradient(loss, [w, b])
    opt.apply_gradients(zip(grads, [w, b]))
    losses.append(float(loss))

print("Estimated w,b:", float(w.numpy().squeeze()), float(b.numpy().squeeze()))

plt.figure(figsize=(6, 4))
plt.plot(losses)
plt.title("SGD with GradientTape")
plt.xlabel("Step")
plt.ylabel("MSE loss")
plt.grid(alpha=0.3)
plt.show()

## Lab plan

An MLP will be trained on Fashion-MNIST. Images will be flattened to 784 features. Early stopping and TensorBoard will be used.

In [ ]:
# Data loading with helper utilities
import tensorflow as tf
from dl_utils import load_tfds_dataset, prepare_for_training, build_callbacks, compile_and_fit, plot_history, visualize_model
from keras import layers, models

# Data
(train_raw, info) = load_tfds_dataset("fashion_mnist", split="train", with_info=True)
test_raw = load_tfds_dataset("fashion_mnist", split="test")
num_classes = info.features["label"].num_classes


def to_float_and_flatten(x, y):
    x = tf.image.convert_image_dtype(x, tf.float32)  # [0,1]
    x = tf.reshape(x, (-1,))                         # 28*28
    return x, y

train_ds = prepare_for_training(train_raw.map(to_float_and_flatten), batch_size=64)
test_ds  = prepare_for_training(test_raw.map(to_float_and_flatten),  batch_size=64, shuffle_buffer=None)

In [ ]:
# Model definition, training, and visualization
model = models.Sequential([
    layers.Input((28*28,)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation="softmax"),
])

visualize_model(model)

callbacks, logdir = build_callbacks(patience=3)
history = compile_and_fit(
    model, train_ds,
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    epochs=5,
    validation_ds=test_ds,
    callbacks=callbacks
)

plot_history(history)
print("To inspect logs locally: tensorboard --logdir runs")

## Concept checks

**Question 1.** What does "flattening" do here?

In [ ]:
if SHOW_SOLUTIONS:
    print("It reshapes each 28x28 image into a vector of 784 numbers.")

**Question 2.** Why does early stopping help?

In [ ]:
if SHOW_SOLUTIONS:
    print("It stops training when validation loss stops improving, avoiding wasted epochs and overfitting.")

**Question 3.** What is the difference between training and validation accuracy?

In [ ]:
if SHOW_SOLUTIONS:
    print("Training accuracy comes from the batches the model learns on, while validation accuracy uses held-out data.")

## Homework

- **Part A (Derivation):** Re-derive the gradient of MSE for $w$ and $b$ in one paragraph.
- **Part B (Code):** Change the hidden size (e.g., 64 or 256) and report the effect on accuracy after 5 epochs.
- **Part C (Analysis):** In 4–6 lines, describe any sign of overfitting or underfitting from the curves.

## Further reading

- TensorFlow: Autodiff basics
- Keras: Image classification guide
- TensorBoard: Scalars and graphs